In [1]:
import os
import numpy as np
from PIL import Image, ImageDraw
from pickle import load
import NNfunctions as NN
from keras.models import load_model

root_dir = 'C:/Users/Varun/Desktop/VOCdevkit/VOC2010'
img_dir = os.path.join(root_dir, 'JPEGImages')

Using TensorFlow backend.


In [2]:
x_master, y_master, z_master = NN.get_dataset('C:/Users/Varun/Desktop/AI-ESA/preprocessing')

[4744, 4744, 4744]


In [3]:
indices = np.arange(x_master.shape[0])
x_master = x_master[indices]
y_master = y_master[indices]
z_master = z_master[indices]

NB_TRAIN = 3500

x_train = np.array(x_master[:NB_TRAIN])
y_train = np.array(y_master[:NB_TRAIN])
z_train = np.array(z_master[:NB_TRAIN])

print(x_train.shape, y_train.shape)

(3500, 128, 128, 3) (3500, 4)


In [4]:
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D


base_model = applications.VGG16(weights = 'imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='relu')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='mse', optimizer='adam', metrics=['mse', 'acc'])

model.summary()

model.fit(x_train, y_train,
          batch_size = 50,
          epochs = 6,
          validation_split = 0.2)

model.save('bounding_box_model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [5]:
model = load_model('bounding_box_mode3.h5')
x_test, y_test = x_master[-1000:], y_master[-1000:]
model.evaluate(x_test, y_test, batch_size = 10)

1000/1000 [==============================] - 32s 32ms/step


[3142.9960498046876, 3142.9960498046876, 0.5540000055730343]

In [12]:
model = load_model('bounding_box_model.h5')
NN.predict_bnd_box(os.path.join(img_dir, '2008_003232.jpg'), model)
del model

[ 24.36787796  29.89999962  89.0993576   89.47468567]
